In [104]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import pyarrow.feather as feather
import matplotlib.pyplot as plt
import numpy as np

In [105]:
filtered_paths = feather.read_feather('Data/dataframes/filtered_paths.feather')

### OTHER IDEA woking with the pairs where there is a ton of samples

We can also make use of the finding that we have over 1000 samples for certain pairs as this will allow for much more precise article scoring metrics. Thus in a third apporach we use only the top occuring start-target pairs and compute and rank the articles that appear using again the same weighted path length - distance approach but also considering if the unfinished paths, penalizing articles that appear in them...

**actually** I am not so sure what this really is so good for. It would be a shame to totally downsample all of these played games. maybe they are quite good for validation. I am thinking that if you we could consider this subnetwork (or only one pair at a time) and do an analysis of article attribute importance to compare to the findings of the global network...

In [106]:
# only consider the top 4 as they all have size > 1000
n = 4
# m = 9 would consider all > 100 

identifier_grouped = filtered_paths.groupby(['identifier'])

# Sort by group size in descending order and take the top 10 largest groups
top_identifiers = identifier_grouped.size().sort_values(ascending=False).head(n).index

# extract all the paths from the filtered df with these identifiers
top_pairs_df = filtered_paths[filtered_paths['identifier'].isin(top_identifiers)].copy()

top_pairs_df.shape

(4868, 15)

In [107]:
# same as in all data let's first look at the articles that appear the most in the paths for reference.
# will use the simplied paths (direct paths) as contrary to full path more apperances necessary imply importance

paths = top_pairs_df['simplified_path'].copy()

paths.apply(lambda l: l[1:-1]) # drop start and target article

article_count = pd.Series(dtype=int)

# Loop through each path and count each article
for path in paths:
    for article in path:
        article_count[article] = article_count.get(article, 0) + 1

# order 
article_count = article_count.sort_values(ascending=False)

article_count.head(10)

Brain               1320
Asteroid            1286
Theatre             1206
Pyramid             1075
Telephone            959
Viking               952
Zebra                802
Earth                715
Africa               573
Computer_science     568
dtype: int64

In [213]:
# function for utils later to get the average weights of articles from a DataFrame containing path information

def calculate_avg_article_weights(df, count_cutoff=30, scaling='standard'):
    """
    Calculate the average weights of articles from a DataFrame containing path information.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'simplified_path': List of articles in the path
            - 'simplified_path_length': Length of the simplified path
            - 'distance': Distance associated with the path
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.
        count_cutoff (int): Minimum number of appearances for an article to be considered

    Returns:
        pd.DataFrame: A DataFrame containing:
            - 'article': Article name
            - 'n_appearances': Number of times the article appeared in paths
            - 'weighted_avg': Weighted average of distances for the article
    """
    # Copy and preprocess the DataFrame
    df = df[['simplified_path', 'simplified_path_length', 'distance']].copy()
    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Calculate weight for each path
    df['weight'] = df['distance'] / df['simplified_path_length']

    # Initialize an empty DataFrame to store results
    avg_article_weight_df = pd.DataFrame(columns=['article', 'n_appearances', 'weighted_avg'])
    avg_article_weight_df.set_index('article', inplace=True)

    # Iterate through each row to calculate weights
    for _, row in df.iterrows():
        weight = row['weight']
        simplified_path = row['simplified_path']

        for article in simplified_path:
            if article not in avg_article_weight_df.index:
                avg_article_weight_df.loc[article] = [0, 0]

            # Update counts and weighted sums
            avg_article_weight_df.at[article, 'n_appearances'] += 1
            avg_article_weight_df.at[article, 'weighted_avg'] += weight

    # Calculate the weighted average by dividing weighted sum by counts
    avg_article_weight_df['weighted_avg'] = avg_article_weight_df['weighted_avg'] / avg_article_weight_df['n_appearances']

    # Filter out articles that appear less than the cutoff
    avg_article_weight_df = avg_article_weight_df[avg_article_weight_df['n_appearances'] >= count_cutoff]

    # Normalize the weighted average
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()

    avg_article_weight_df[scaling] = scaler.fit_transform(avg_article_weight_df[['weighted_avg']])


    print(f"Number of unique articles after weighting: {avg_article_weight_df.shape[0]}")

    return avg_article_weight_df#.reset_index()


In [215]:
avg_article_w_top_pais = calculate_avg_article_weights(top_pairs_df[top_pairs_df['finished']])
avg_article_w_top_pais.sort_values(by='weighted_avg', ascending=False).head(10)

/var/folders/sj/219f9qbn3y15yynshcyrrp3c0000gn/T/ipykernel_17178/378903692.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.42857142857142855' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  avg_article_weight_df.at[article, 'weighted_avg'] += weight


Number of unique articles after weighting: 92


,n_appearances,weighted_avg,standard
article,,,
India,201,0.751297,2.723717
Carl_Friedrich_Gauss,92,0.724573,2.361273
19th_century,37,0.721686,2.322118
Psychology,66,0.710209,2.166466
World_Wide_Web,60,0.694226,1.949697
Information,249,0.691050,1.906620
Dance,109,0.688161,1.867436
Communication,496,0.679757,1.753462
Denmark,162,0.651595,1.371511


In [163]:
# code a function that returns the ratio of the number of times an article appears in unfinished paths over the total number of times it appears

def ratio_unfinished(in_df, count_cutoff=30, scaling='standard'):
    """
    Calculate the ratio of the number of times an article appears in unfinished paths over the total number of times it appears.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'simplified_path': List of articles in the path
        count_cutoff (int): Minimum number of appearances for an article to be considered
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.

    Returns:
        pd.Series: A Series containing the ratio for each article
    """
    # Copy and preprocess the DataFrame
    df = in_df[['simplified_path', 'finished']].copy()
    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Initialize a dictionary to store counts
    article_counts = {}
    unfinished_counts = {}

    # Iterate through each row to calculate counts
    for _, row in df.iterrows():
        simplified_path = row['simplified_path']
        finished = row['finished']

        for article in simplified_path:
            article_counts[article] = article_counts.get(article, 0) + 1
        
        if not finished:
            for article in simplified_path:
                unfinished_counts[article] = unfinished_counts.get(article, 0) + 1

    # Convert the dictionary to a Series
    article_counts = pd.Series(article_counts)
    unfinished_counts = pd.Series(unfinished_counts)

    ratio = unfinished_counts / article_counts

    ratio_df = pd.DataFrame({
    'n_appearances': article_counts,
    'unfinished_counts': unfinished_counts,
    'unfinished_ratio': ratio
    }).fillna(0)

    # cut off
    ratio_df = ratio_df[ratio_df['n_appearances'] >= count_cutoff]

    # scaling
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()
    
    ratio_df[scaling] = scaler.fit_transform(ratio_df[['unfinished_ratio']])

    #print(f"Number of unique articles: {len(article_counts)}")
    print(f"Ratio of unfinished over finished paths: {1-df['finished'].mean()}")
    return ratio_df

In [157]:
ratio_unfinished_top_pairs = ratio_unfinished(top_pairs_df).sort_values(by='unfinished_ratio' , ascending=False)

ratio_unfinished_top_pairs[ratio_unfinished_top_pairs['n_appearances'] >= 30]

Number of unique articles: 1005
Ratio of unfinished over finished paths: 0.3278553820870994


,n_appearances,unfinished_counts,unfinished_ratio,StandardScaler()
Salsa_music,34,25.0,0.735294,2.522854
Latin_America,32,22.0,0.687500,2.244719
Protein,50,32.0,0.640000,1.968296
Mexico_City,70,43.0,0.614286,1.818653
Irrigation,50,30.0,0.600000,1.735518
...,...,...,...,...
Pea,92,1.0,0.010870,-1.692893
Electronics,143,1.0,0.006993,-1.715453
Alexander_Graham_Bell,47,0.0,0.000000,-1.756148
Electricity,71,0.0,0.000000,-1.756148


In [202]:
# code a function that counts the number of dead ends an article has (difference between full path list content and simplified path list content)

def calculate_detour_ratios(in_df, count_cutoff=1, scaling='standard'):
    """
    Calculate the detour ratio for articles based on the full path and simplified path.

    Parameters:
        in_df (pd.DataFrame): Input DataFrame with the following columns:
            - 'full_path': List of articles in the full path
            - 'simplified_path': List of articles in the simplified path
        count_cutoff (int): Minimum number of detours for an article to be considered.
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.

    Returns:
        pd.DataFrame: A DataFrame containing the detour ratio and scaled values for each article.
    """
    # Copy and preprocess the DataFrame
    df = in_df[['full_path', 'simplified_path']].copy()
    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles
    df['full_path'] = df['full_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Initialize dictionaries to store counts
    detour_counts = {}
    total_counts = {}

    # Iterate through each row to calculate detour counts and total appearances
    for _, row in df.iterrows():
        full_path = row['full_path']
        simplified_path = row['simplified_path']

        # Count total appearances for articles in the full path
        for article in full_path:
            total_counts[article] = total_counts.get(article, 0) + 1

        # Find detour articles by subtracting the simplified path from the full path
        detour_articles = set(full_path) - set(simplified_path)
        for article in detour_articles:
            detour_counts[article] = detour_counts.get(article, 0) + 1

    # Convert counts to Series
    detour_counts = pd.Series(detour_counts)
    total_counts = pd.Series(total_counts)

    # Fill missing detour counts with 0 for articles with no detours
    detour_counts = detour_counts.reindex(total_counts.index, fill_value=0)

    # Calculate detour ratio
    detour_ratios = detour_counts / total_counts

    # Create a DataFrame with detour counts and ratios
    detour_df = pd.DataFrame({
        'detour_count': detour_counts,
        'total_count': total_counts,
        'detour_ratio': detour_ratios
    }).loc[detour_ratios.index]

    # Filter out articles with detour ratio less than the count_cutoff
    detour_df = detour_df[detour_df['total_count'] >= count_cutoff]

    # Normalize the detour ratios
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()

    detour_df[scaling] = scaler.fit_transform(detour_df[['detour_ratio']])

    print(f"Number of unique articles after detour ratio calculation: {len(detour_df)}")
    return detour_df

# it is not really necessary to filter for articles that appear more than a certain amount of times since we do it in the other scores anyway and the combined df will thus be fine

In [205]:
detour_couts_top_pairs = calculate_detour_ratios(top_pairs_df, count_cutoff=30, scaling='minmax').sort_values(by='detour_count', ascending=False)
detour_couts_top_pairs

Number of unique articles after detour ratio calculation: 137


,detour_count,total_count,detour_ratio,minmax
Herbivore,59,197,0.299492,0.633542
Soybean,58,310,0.187097,0.395782
Protein,31,82,0.378049,0.799719
Optical_fiber,26,55,0.472727,1.000000
Coffee,23,52,0.442308,0.935651
...,...,...,...,...
Telecommunication,0,65,0.000000,0.000000
Electronics,0,143,0.000000,0.000000
Fruit,0,63,0.000000,0.000000
Vegetable,0,81,0.000000,0.000000


Comment:
this can be a penalty term: the smart way to implement it is by just a normalized version. 

why not immediately consider the full_path - distance metric => because it penalizes all articles in the path but only the 'detour' articles should be penalized.

thus, the best way to do it is to combine the computed distance_difference metric, with this penaly term, and maybe the unfinished ratio penalty as well.

In [114]:
# combine the metrics into a composite score => do a df with the articles as index and the metrics as columns, and their composite as a new column

In [115]:
# check that not all most played paths paths are played by the same players
top_pairs_df['hashedIpAddress'].unique().shape


(3107,)

In [116]:
# just to be sure
id1 = 6500.0
id2 = 3433.0
id3 = 38072.0 
id4 = 31392.0

print("top 1: ", top_pairs_df[top_pairs_df['identifier'] == id1]['hashedIpAddress'].value_counts().max())
print("top 2: ", top_pairs_df[top_pairs_df['identifier'] == id2]['hashedIpAddress'].value_counts().max())
print("top 3: ", top_pairs_df[top_pairs_df['identifier'] == id3]['hashedIpAddress'].value_counts().max())
print("top 4: ", top_pairs_df[top_pairs_df['identifier'] == id4]['hashedIpAddress'].value_counts().max())


top 1:  14
top 2:  10
top 3:  11
top 4:  11


# Some playing with time aspect

In [185]:
finished_paths = filtered_paths[filtered_paths['finished']]

# Initialize a dictionary to store the DataFrames
dfs_by_distance = {}

# Iterate over the range of distances
for d in range(1, int(finished_paths['distance'].max()) + 1):
    # Filter and sort the DataFrame for the current distance
    df_d = finished_paths[finished_paths['distance'] == d].sort_values(by='durationInSec')
    
    # Store the DataFrame in the dictionary with the key `d`
    dfs_by_distance[f"df_{d}"] = df_d

# Display the first few rows of the DataFrame for distance d
d = 3
top = dfs_by_distance[f"df_{d}"].head(20)
print(top['hashedIpAddress'].unique().shape)
print(top['identifier'].unique().shape)
top.head(20)

# NOTE: probably two friends challenged themselves to get this one path as fast as possible


(7,)
(7,)


,hashedIpAddress,timestamp,durationInSec,path,rating,finished,failure_reason,start_article,target_article,identifier,full_path,simplified_path,full_path_length,simplified_path_length,distance
2277,3c4bf61c4a447176,1345664849,7,Bird;Fish;Whale_shark;Basking_shark;Great_whit...,NaN,True,None,Bird,Great_white_shark,5634,"[Bird, Fish, Whale_shark, Basking_shark, Great...","[Bird, Fish, Whale_shark, Basking_shark, Great...",4,4,3.0
2290,62d5c0be1ee5c287,1349221060,7,Bird;Fish;Whale_shark;Shark;Great_white_shark,NaN,True,None,Bird,Great_white_shark,5634,"[Bird, Fish, Whale_shark, Shark, Great_white_s...","[Bird, Fish, Whale_shark, Shark, Great_white_s...",4,4,3.0
24917,0d57c8c57d75e2f5,1294612021,7,Wernher_von_Braun;Physics;Atom;Helium,1.0,True,None,Wernher_von_Braun,Helium,41318,"[Wernher_von_Braun, Physics, Atom, Helium]","[Wernher_von_Braun, Physics, Atom, Helium]",3,3,3.0
2273,3c4bf61c4a447176,1345664725,7,Bird;Fish;Whale_shark;Basking_shark;Great_whit...,NaN,True,None,Bird,Great_white_shark,5634,"[Bird, Fish, Whale_shark, Basking_shark, Great...","[Bird, Fish, Whale_shark, Basking_shark, Great...",4,4,3.0
22654,7d5624e35c9523ef,1370030125,7,Gunpowder;Sulfur;Periodic_table;Ytterbium,NaN,True,None,Gunpowder,Ytterbium,17222,"[Gunpowder, Sulfur, Periodic_table, Ytterbium]","[Gunpowder, Sulfur, Periodic_table, Ytterbium]",3,3,3.0
28733,4e8e0c2b0b490fb5,1298903608,8,Meteorological_history_of_Hurricane_Katrina;Un...,NaN,True,None,Meteorological_history_of_Hurricane_Katrina,Iraq_War,25246,"[Meteorological_history_of_Hurricane_Katrina, ...","[Meteorological_history_of_Hurricane_Katrina, ...",3,3,3.0
2275,3c4bf61c4a447176,1345664777,8,Bird;Fish;Whale_shark;Basking_shark;Great_whit...,NaN,True,None,Bird,Great_white_shark,5634,"[Bird, Fish, Whale_shark, Basking_shark, Great...","[Bird, Fish, Whale_shark, Basking_shark, Great...",4,4,3.0
8450,614f1f9a3c5772c5,1252572105,8,Wikipedia;Amsterdam;Netherlands;Cheese,NaN,True,None,Wikipedia,Cheese,41546,"[Wikipedia, Amsterdam, Netherlands, Cheese]","[Wikipedia, Amsterdam, Netherlands, Cheese]",3,3,3.0
2266,3c4bf61c4a447176,1345664490,8,Bird;Fish;Whale_shark;Basking_shark;Great_whit...,NaN,True,None,Bird,Great_white_shark,5634,"[Bird, Fish, Whale_shark, Basking_shark, Great...","[Bird, Fish, Whale_shark, Basking_shark, Great...",4,4,3.0
7410,614f1f9a3c5772c5,1252571942,8,Shark;Australia;Football_(soccer);FIFA,5.0,True,None,Shark,FIFA,34267,"[Shark, Australia, Football_(soccer), FIFA]","[Shark, Australia, Football_(soccer), FIFA]",3,3,3.0


In [128]:
finished_paths[finished_paths['identifier'] == 5634]['hashedIpAddress'].value_counts().head()

hashedIpAddress
5f22535607b1d77d    29
597c31a9232d63ef    13
3c4bf61c4a447176    13
62d5c0be1ee5c287    12
7d5624e35c9523ef    12
Name: count, dtype: int64

In [138]:
finished_paths.sort_values(by='durationInSec',  ascending=False).head(5)

,hashedIpAddress,timestamp,durationInSec,path,rating,finished,failure_reason,start_article,target_article,identifier,full_path,simplified_path,full_path_length,simplified_path_length,distance
10855,3c623ddc539a389a,1262371920,35091,Norse_mythology;Literature;Fiction;Computer_an...,NaN,True,None,Norse_mythology,Card_game,27738,"[Norse_mythology, Literature, Fiction, Compute...","[Norse_mythology, Literature, Fiction, Compute...",7,7,3.0
49073,605d30402bc1fed5,1371354294,34009,Theatre;United_States;Pacific_Ocean;Tuna;Fish;...,NaN,True,None,Theatre,Zebra,38072,"[Theatre, United_States, Pacific_Ocean, Tuna, ...","[Theatre, United_States, Pacific_Ocean, Tuna, ...",7,7,3.0
23509,0be646af2bbb2afe,1326699667,23326,Joan_of_Arc;Hundred_Years'_War;Weapon;Nuclear_...,2.0,True,None,Joan_of_Arc,Nuclear_fission,20988,"[Joan_of_Arc, Hundred_Years'_War, Weapon, Nucl...","[Joan_of_Arc, Hundred_Years'_War, Weapon, Nucl...",4,4,3.0
23023,38f003597665573a,1380149197,19740,Church_of_England;England;Charles_Darwin;Biolo...,3.0,True,None,Church_of_England,Nitrogen_fixation,8906,"[Church_of_England, England, Charles_Darwin, B...","[Church_of_England, England, Charles_Darwin, B...",10,10,4.0
43583,6fbaf91828a2dfe0,1368860318,17171,Music_of_Ireland;Ireland;Europe;Germany;German...,NaN,True,None,Music_of_Ireland,Merseburg_Incantations,26576,"[Music_of_Ireland, Ireland, Europe, Germany, G...","[Music_of_Ireland, Ireland, Europe, Germany, G...",5,5,3.0


In [186]:
# NOTE: if we want to use speed we also need to remove speed outliers 
# the best way to do it is probably using the IQR method but for each shortest distance seperately...

def filter_duration(df):
    """
    Filter the DataFrame based on the distance and duration bounds using the IQR method. And downsample to one IpAdress per identifier.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'distance': Distance associated with the path
            - 'durationInSec': Duration associated with the path

    Returns:
        pd.DataFrame: Filtered DataFrame
    """
    filtered_dfs = []  # List to hold filtered data for each distance group

    for d in range(1, int(df['distance'].max()) + 1):
        # Filter the DataFrame for the current distance group
        df_d = df[df['distance'] == d]

        # Compute IQR for 'durationInSec'
        Q1 = df_d['durationInSec'].quantile(0.25)
        Q3 = df_d['durationInSec'].quantile(0.75)
        IQR = Q3 - Q1

        # Calculate upper bound based on IQR
        upper_bound = Q3 + 1.5 * IQR

        # Keep only rows within the upper bound
        filtered_df_d = df_d[df_d['durationInSec'] <= upper_bound]

        # Append filtered group to the list
        filtered_dfs.append(filtered_df_d)

    # Concatenate all filtered groups
    filtered_df = pd.concat(filtered_dfs, ignore_index=True)
    
    # downsample data to one IpAdress per identifier
    downsampled_df = filtered_df.groupby(['hashedIpAddress', 'identifier']).sample(n=1, random_state=42)

    # Calculate the number of removed rows
    removed = df.shape[0] - downsampled_df.shape[0]

    # Print the result
    print(f"In sampling a total of {removed} samples were removed, "
        f"which represents {removed / df.shape[0] * 100:.3f}% of the original data.",
        f"{df.shape[0]} samples remain.")

    return downsampled_df

finished_paths = filter_duration(finished_paths)
    

In sampling a total of 4713 samples were removed, which represents 9.884% of the original data. 47683 samples remain.


In [187]:
# check if the filtering worked
finished_paths.sort_values(by='durationInSec', ascending=False)

,hashedIpAddress,timestamp,durationInSec,path,rating,finished,failure_reason,start_article,target_article,identifier,full_path,simplified_path,full_path_length,simplified_path_length,distance
45030,104ea212055eb19e,1248675515,496,Soil_profile;Soil;<;Soil;Yellow_River;China;Wo...,2.0,True,None,Soil_profile,Michigan_State_University,34982,"[Soil_profile, Soil, Soil, Yellow_River, China...","[Soil_profile, Soil, Yellow_River, China, Worl...",11,7,5.0
44920,39de3a9b41f563cc,1351819671,492,Blitzkrieg;American_football;United_States;Atl...,NaN,True,None,Blitzkrieg,Tropical_Storm_Delta_(2005),5933,"[Blitzkrieg, American_football, United_States,...","[Blitzkrieg, American_football, United_States,...",10,10,5.0
44742,067ebbe12b292e09,1347305965,476,First_Macedonian_War;Ancient_Greece;Continent;...,3.0,True,None,First_Macedonian_War,2003_Atlantic_hurricane_season,14353,"[First_Macedonian_War, Ancient_Greece, Contine...","[First_Macedonian_War, Ancient_Greece, Contine...",8,8,5.0
44896,20c9136f7a7fddc7,1310502341,464,List_of_French_monarchs;France;European_Union;...,NaN,True,None,List_of_French_monarchs,Dunnock,23006,"[List_of_French_monarchs, France, European_Uni...","[List_of_French_monarchs, France, European_Uni...",10,10,5.0
44890,4d9a393e486623e3,1269374123,462,Kookaburra;Australia;France;European_Union;Gre...,3.0,True,None,Kookaburra,Battle_of_Lechaeum,21821,"[Kookaburra, Australia, France, European_Union...","[Kookaburra, Australia, France, European_Union...",7,7,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,654fc5e419e5c364,1269452944,2,Apple;Fruit,NaN,True,None,Apple,Fruit,2835,"[Apple, Fruit]","[Apple, Fruit]",1,1,1.0
327,614f1f9a3c5772c5,1252572197,2,Wikipedia;Amsterdam,5.0,True,None,Wikipedia,Amsterdam,41543,"[Wikipedia, Amsterdam]","[Wikipedia, Amsterdam]",1,1,1.0
627,63626e6f109a1b46,1371342811,2,Adolf_Hitler;Germany,NaN,True,None,Adolf_Hitler,Germany,1066,"[Adolf_Hitler, Germany]","[Adolf_Hitler, Germany]",1,1,1.0
545,2cc27ed1036059f0,1355975823,2,Food;Fishing,1.0,True,None,Food,Fishing,14684,"[Food, Fishing]","[Food, Fishing]",1,1,1.0


In [ ]:
def calc_avg_article_time(df, count_cutoff=30, scaling='standard'):
    """
    Calculate the average speed of articles from a DataFrame containing path information.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'simplified_path': List of articles in the path
            - 'durationInSec': Duration associated with the path
        count_cutoff (int): Minimum number of appearances for an article to be considered
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.

    Returns:
        pd.DataFrame: A DataFrame containing:
            - 'article': Article name
            - 'n_appearances': Number of times the article appeared in paths
            - 'avg_speed': Average speed of the article
    """
    # Copy and preprocess the DataFrame
    df = df[['simplified_path', 'durationInSec']].copy()

    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Initialize an empty DataFrame to store results
    avg_article_speed_df = pd.DataFrame(columns=['article', 'n_appearances', 'avg_speed'])
    avg_article_speed_df.set_index('article', inplace=True)

    # Iterate through each row to calculate speeds
    for _, row in df.iterrows():
        speed = row['durationInSec']
        simplified_path = row['simplified_path']

        for article in simplified_path:
            if article not in avg_article_speed_df.index:
                avg_article_speed_df.loc[article] = [0, 0]

            # Update counts and sums
            avg_article_speed_df.at[article, 'n_appearances'] += 1
            avg_article_speed_df.at[article, 'avg_speed'] += speed

    # Calculate the average speed by dividing sum by counts
    avg_article_speed_df['avg_speed'] = avg_article_speed_df['avg_speed'] / avg_article_speed_df['n_appearances']

    # Filter out articles that appear less than the cutoff
    avg_article_speed_df = avg_article_speed_df[avg_article_speed_df['n_appearances'] >= count_cutoff]

    # Normalize the average speed
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()
    
    avg_article_speed_df[scaling] = scaler.fit_transform(avg_article_speed_df[['avg_speed']])

    print(f"Number of unique articles after time calc: {avg_article_speed_df.shape[0]}")

    return avg_article_speed_df#.reset_index()


# COMMENT: could consider really computing the speed instead of the duration. speed = distance / time and then sum up and average.

In [188]:
avg_article_speed_top_pais = calc_avg_article_time(finished_paths).sort_values(by='avg_speed')

avg_article_speed_top_pais[avg_article_speed_top_pais['n_appearances'] >= 30]

Number of unique articles after time calc: 777


,n_appearances,avg_speed,standard
article,,,
North_Korea,41,84.780488,-3.005867
Old_English_language,39,87.923077,-2.822119
Korea,42,90.238095,-2.686760
Suez_Canal,38,94.263158,-2.451414
Portugal,144,99.520833,-2.143996
...,...,...,...
Space_Race,45,189.800000,3.134643
2005_Atlantic_hurricane_season,61,191.524590,3.235480
Extrasolar_planet,63,195.095238,3.444257


In [ ]:

# before I do that I ll talk about it with the team
finished_paths['durationInSec'].describe()

# downsample data to one IpAdress per identifier
downsampled_df = finished_paths.groupby(['hashedIpAddress', 'identifier']).sample(n=1, random_state=42)

# Calculate the number of removed rows
removed = finished_paths.shape[0] - downsampled_df.shape[0]

# Print the result
print(f"In sampling a total of {removed} samples were removed, "
      f"which represents {removed / finished_paths.shape[0] * 100:.3f}% of the original data.",
      f"{finished_paths.shape[0]} samples remain.")


avg_article_speed_top_pais = calc_avg_article_time(downsampled_df).sort_values(by='avg_speed')

avg_article_speed_top_pais[avg_article_speed_top_pais['n_appearances'] >= 30]


In sampling a total of 2108 samples were removed, which represents 4.676% of the original data. 45078 samples remain.
Number of unique articles: 777


,n_appearances,avg_speed,standard
article,,,
North_Korea,41,84.780488,-3.005867
Old_English_language,39,87.923077,-2.822119
Korea,42,90.238095,-2.686760
Suez_Canal,38,94.263158,-2.451414
Portugal,144,99.520833,-2.143996
...,...,...,...
Space_Race,45,189.800000,3.134643
2005_Atlantic_hurricane_season,61,191.524590,3.235480
Extrasolar_planet,63,195.095238,3.444257


## make a composite df with all the different scores

In [ ]:
finished_paths = filtered_paths[filtered_paths['finished']]

# downsample data to one IpAdress per identifier
# this way players can't just learn paths and then play them as fast as possible
finished_paths = finished_paths.groupby(['hashedIpAddress', 'identifier']).sample(n=1, random_state=42)

weight_df = calculate_avg_article_weights(finished_paths, count_cutoff=30, scaling='standard')
time_df = calc_avg_article_time(filter_duration(finished_paths), count_cutoff=30, scaling='standard')
unfinished_atio_df = ratio_unfinished(filtered_paths, count_cutoff=30, scaling='standard')
detour_ratio_df = calculate_detour_ratios(finished_paths, count_cutoff=30, scaling='standard')

/var/folders/sj/219f9qbn3y15yynshcyrrp3c0000gn/T/ipykernel_17178/378903692.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  avg_article_weight_df.at[article, 'weighted_avg'] += weight


Number of unique articles after weighting: 820
In sampling a total of 2471 samples were removed, which represents 5.437% of the original data. 45451 samples remain.
Number of unique articles after time calc: 776
Ratio of unfinished over finished paths: 0.1762317738926128
Number of unique articles after detour ratio calculation: 871


In [223]:
# Combine the metrics into a composite score
composite_df = pd.DataFrame(index=weight_df.index)
composite_df['weighted_avg'] = weight_df['standard']
composite_df['avg_speed'] = time_df['standard']
composite_df['unfinished_ratio'] = unfinished_atio_df['standard']
composite_df['detour_ratio'] = detour_ratio_df['standard']

composite_df

# | article | weighted_avg | avg_speed | unfinished_ratio | detour_ratio |
# |---------|bigger better |small better|   small better  | small better |
#   

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
Philosophy,0.924846,-0.245149,-0.213956,-0.146164
Mathematics,0.893903,-0.200423,-0.896721,-1.029241
Arithmetic,1.100646,0.575692,-0.953803,0.221156
North_Africa,-0.605665,0.060291,0.090040,-0.155223
Africa,0.907139,-0.828117,-0.529661,-0.973283
...,...,...,...,...
United_States_Senate,-1.257078,-1.575148,1.073610,2.311367
Cheese,2.737321,NaN,-0.492802,-0.644868
Nobel_Peace_Prize,0.919959,-0.610105,-0.569058,-0.193466


In [224]:
composite_df.sort_values(by='weighted_avg', ascending=False)

# comment: maybe not only downsample to unique player per identifier for speed but also for the other metrics => include it in the normal filtering pipeline

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
Achilles,4.811790,-2.045308,-0.850784,-1.312944
J._K._Rowling,4.044191,-1.277557,-1.006611,-0.949604
Mario,3.390951,0.830807,-0.859010,-0.277426
Harry_Potter,3.188972,-0.005016,-0.565727,-1.312944
Lead,3.156074,NaN,-0.492802,3.047129
...,...,...,...,...
Anatomy,-2.276821,0.853373,2.486595,0.569815
Irrigation,-2.391556,-0.076705,1.989156,0.646143
Gas,-2.539718,-0.609033,0.473747,3.601376


In [225]:
composite_df.sort_values(by='avg_speed')

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
North_Korea,0.431010,-3.003798,0.153795,0.890285
Old_English_language,0.054290,-2.821436,1.878974,0.487956
Korea,1.232629,-2.687098,-0.748606,-0.842254
Suez_Canal,0.080033,-2.453527,-0.979452,1.223017
President_of_the_United_States,1.664244,-2.117642,-1.205954,-0.936918
...,...,...,...,...
Welding,1.771787,NaN,-0.542058,-1.312944
List_of_rivers_by_length,1.915145,NaN,-1.006611,-0.018547
Oxford,-0.075141,NaN,0.456928,1.957111


In [226]:
composite_df.sort_values(by='detour_ratio')

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
Welding,1.771787,NaN,-0.542058,-1.312944
Coin,0.816901,-0.373638,-1.559253,-1.312944
List_of_countries,0.528985,-0.741666,-2.096106,-1.312944
United_States_Congress,-0.491675,NaN,0.521052,-1.312944
"Detroit,_Michigan",0.848160,3.627169,-0.748606,-1.312944
...,...,...,...,...
Yellowstone_National_Park,-0.989639,1.589729,2.118855,3.289356
Eukaryote,-1.550714,0.361369,2.631404,3.433178
Gas,-2.539718,-0.609033,0.473747,3.601376


## Some thoughts

what makes a good article in the game?
we have so far left aside the time aspect but this actually is part of the name 'speedia'

- can rate a path based on normalized its speed (normalized to all the finished apths of that distance)
  Here it is important that each start-target pair is only played once per player (hashedId)
- can rate a path based on difference to optimal path length. Here it can make sense to incorporate the back
- a combination of both!


**So least amount of clicks vs speed!!**

okay and then try to find out what article attributes are more related to one or the other?

In [ ]:
unfinished = filtered_paths[filtered_paths['finished'] == False]

unfinished['failure_reason'].value_counts()

# Comment could maybe considering filtering out timetouts as well?

failure_reason
restart    6571
timeout    3630
Name: count, dtype: int64